In [1]:
import pandas as pd
from datetime import datetime, timedelta
from analyzer.live_entry_strats import LiveEntryStrats as les
from analyzer.live_exit_strats import LiveExitStrats as lxs
from time import sleep
from database.comet import Comet
from coinbase.coinbase import Coinbase as cbs
from processor.processor import Processor as p
import pytz

In [2]:
comet = Comet()

In [3]:
comet.cloud_connect()
historicals = comet.retrieve("cloud_test_coinbase_hourly")
trading_params = comet.retrieve("btc_trading_params")
comet.disconnect()

In [4]:
trading_params

,signal,req,trades,pv,days,retrack_days,value,conservative,entry_strategy,exit_strategy
0,0.01,0.05,11,285.311671,12.272727,3,False,True,standard,hold


In [5]:
offerings = les.entry_analysis(
            trading_params["entry_strategy"].item()
           ,historicals
           ,trading_params["signal"].item()
           ,trading_params["value"].item()
           ,trading_params["conservative"].item()
            )

In [6]:
accounts = cbs.get_accounts()
balance = accounts[accounts["currency"]=="USD"]["balance"].iloc[0]
pending_orders = cbs.get_orders()
pending_orders = p.live_column_date_processing(pending_orders.rename(columns={"created_at":"date"}))
fls = []
status = "fills"
for currency in accounts["currency"].unique():
    fill = cbs.get_fill(currency)
    if len(fill) > 0:
        try:
            f = pd.DataFrame(fill)
            fls.append(f)
        except:
            continue
## sells
if len(fls) > 0:
    fills = pd.concat(fls)
    #store_non_existing_executed_buy
    existing_fills = comet.retrieve_fills()
    if existing_fills.index.size > 1:
        existing_order_ids = list(existing_fills["order_id"])
    else:
        existing_order_ids = []
    new_fills = fills[~fills["order_id"].isin(existing_order_ids)]        
    if new_fills.index.size > 0:
        incomplete_trades = new_fills[(new_fills["side"]=="buy")]
        incomplete_trades["size"] = [float(x) for x in incomplete_trades["size"]]
        incomplete_trades["price"] = [float(x) for x in incomplete_trades["price"]]
        for oi in incomplete_trades["order_id"].unique():
            order_trades = incomplete_trades[incomplete_trades["order_id"]==oi]
            if len([x for x in order_trades["settled"] if x == False]) == 0 and order_trades.index.size > 0:
                incomplete_trade = lxs.exit_analysis(exit_strategy,merged,order_trades,retrack_days,req)
                if "sell_price" in incomplete_trade.keys():
                    sell_statement = cbs.place_sell(incomplete_trade["product_id"]
                                                    ,incomplete_trade["sell_price"]
                                                    ,incomplete_trade["size"])
                    incomplete_trade["sell_id"] = sell_statement["id"]

In [7]:
if new_fills.index.size > 0:
    completed_trades = new_fills[(new_fills["side"]=="sell")]
    for soi in completed_trades["order_id"].unique():
        sell_order_trades = completed_trades[completed_trades["order_id"]==soi]
        if len([x for x in sell_order_trades["settled"] if x == False]) == 0:
            complete_trade = sell_order_trades.iloc[0]
            order_id = complete_trade["order_id"]
            one_half = comet.retrieve_incomplete_trade(order_id)
            one_half["sell_date"] = complete_trade["created_at"]
            one_half["sell_price"] = complete_trade["price"]

In [10]:
new_fills

,created_at,trade_id,product_id,order_id,user_id,profile_id,liquidity,price,size,fee,side,settled,usd_volume
